# SSO + Snowpark Tester

### To sign into snowpark, you need to have login info. but with sos, we have sso so i need to try logging in that way

In [22]:
import numpy as np
import pandas as pd
import json

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Use pathlib to get the current working directory, 
# then cd.. into the parent folder where the snowpark_credientials.json
# is located.
from pathlib import Path
file_path_split = str(Path.cwd()).split("/")
parent_folder = "/".join(file_path_split)

with open(parent_folder + "/sso_credentials.json", "r") as f:
    connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [23]:
t = session.table("HUBSPOT__CONTACTS")
t.to_pandas()

,CONTACT_ID,IS_CONTACT_DELETED,EMAIL,CONTACT_COMPANY,FIRST_NAME,LAST_NAME,CREATED_AT,JOB_TITLE,COMPANY_ANNUAL_REVENUE,_FIVETRAN_SYNCED,...,TOTAL_UNIQUE_SPAM_REPORTS,TOTAL_UNSUBSCRIBES,TOTAL_UNIQUE_UNSUBSCRIBES,COUNT_ENGAGEMENT_NOTES,COUNT_ENGAGEMENT_TASKS,COUNT_ENGAGEMENT_CALLS,COUNT_ENGAGEMENT_MEETINGS,COUNT_ENGAGEMENT_EMAILS,COUNT_ENGAGEMENT_INCOMING_EMAILS,COUNT_ENGAGEMENT_FORWARDED_EMAILS
0,245592904,False,pmarchand@mac.com,Biocept,Philippe,Marchand,2022-10-26 14:31:47.790000-07:00,COO,2.5995E7,2023-04-14 22:43:40.494000-07:00,...,0,0,0,0,0,0,0,0,0,0
1,240864501,False,diyan.bogdanov@payhawk.com,Payhawk,Diyan,Bogdanov,2022-10-03 07:55:52.794000-07:00,None,None,2023-04-14 10:33:51.271000-07:00,...,0,0,0,0,0,0,0,0,0,0
2,189190998,False,shannon.webster@firstcitizens.com,None,Shannon,Webster,2022-05-13 16:07:16.093000-07:00,"Senior Vice President/Officer and Director, IT...",None,2023-04-14 10:24:11.073000-07:00,...,0,0,0,0,0,0,0,0,0,0
3,6824851,False,gpolaki@wintrust.com,None,Giridhar,Polaki,2020-02-26 10:47:21.179000-08:00,Lead Data Architect,None,2023-04-14 10:15:16.711000-07:00,...,0,0,0,0,0,0,0,0,0,0
4,129930276,False,ksnyder@bokf.com,BOK Financial Corporation,Kylee,Snyder,2021-10-03 09:48:12.034000-07:00,Vice President Customer Experience,None,2023-04-14 10:19:03.477000-07:00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888405,189203962,False,oscar.pena@savemart.com,None,Oscar,Pena,2022-05-13 16:07:17.485000-07:00,"Senior Director, Pricing & Promotions",None,2023-04-14 10:24:25.051000-07:00,...,0,0,0,0,0,0,0,0,0,0
888406,130615354,False,laura.finlayson@alteryx.com,"Alteryx, Inc.",Laura,Finlayson,2021-10-17 14:18:44.408000-07:00,Director communications,None,2023-04-14 10:19:08.865000-07:00,...,0,0,0,0,0,0,0,0,0,0
888407,44082053,False,mark.govostes@wdc.com,Western Digital,Mark,Govostes,2020-10-09 12:20:35.179000-07:00,Director Data Warehouse & Business Intelligence,None,2023-04-20 14:43:34.176000-07:00,...,0,0,0,0,0,0,0,0,0,0
888408,43595477,False,rconnolly@indeed.com,None,Reid,Connolly,2020-10-08 11:34:25.173000-07:00,"Manager, Business Intelligence",None,2023-04-20 14:45:07.953000-07:00,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Register this as vectorized function that we can reference in Sigma

def cron_descriptor_to_readable_text_udf(cron_input: str) -> dict:

    from cron_descriptor import get_description
    desc = get_description(cron_input)

    if desc.find(",") == -1:
        desc = desc + ", daily"

    return {'cron_input': cron_input, 'cron_readable': desc}


# Register Vectorized UDF
cron_descriptor_to_readable_text_udf = session.udf.register(func=cron_descriptor_to_readable_text_udf,
                                                        name="CRON_DESCRIPTOR_TO_READABLE_TEXT_UDF",
                                                        stage_location='@MODELS', # can point to any stage in the schema, doesn't mattter which but needs to be included
                                                        input_types=[T.StringType()],
                                                        return_type=T.VariantType(),
                                                        replace=True,
                                                        is_permanent=True,
                                                        packages=['pandas', 'cron-descriptor'],
                                                        max_batch_size=1000,
                                                        session=session)


The version of package cron-descriptor in the local environment is 1.2.35, which does not fit the criteria for the requirement cron-descriptor. Your UDF might not work when the package version is different between the server and your local environment
